In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os 
from pyod.models.iforest import IForest
import numpy as np
from sklearn.metrics import classification_report
from pyod.models.hbos import HBOS
from sklearn.metrics import accuracy_score



In [2]:
df=pd.read_csv("/Users/saikarthik/Desktop/Nexus-S1 Deakin /cleaned.csv")

In [3]:
mean_error = df["value"].mean()
std_error = df["value"].std()


In [4]:
thresholds = [3,4]  

df['true_labels'] = 0

for threshold_mult in thresholds:
    threshold_value = mean_error + threshold_mult * std_error
    df['true_labels'] = np.where(df['value'] > threshold_value, 1, df['true_labels'])

In [5]:
df["true_labels"].value_counts()

true_labels
0    22695
Name: count, dtype: int64

## Isolation Forest 

In [6]:
X = df[['value']].values


In [7]:
clf = IForest(contamination=0.1, random_state=42)

In [8]:
clf.fit(X)


IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=42,
    verbose=0)

In [9]:
y_pred_if = clf.predict(X) 

In [10]:
y_scores_if= clf.decision_function(X)

In [11]:
plt.figure(figsize=(12, 6))
plt.plot(df['timestamp'], df['value'], label='Time Series Data')
plt.scatter(df['timestamp'][y_pred_if== 1], df['value'][y_pred_if== 1], 
            color='red', marker='x', label='Detected Anomalies (HBOS)')
plt.scatter(df['timestamp'][df['true_labels'] == 1], df['value'][df['true_labels'] == 1], 
            color='blue', marker='o', label='Ground Truth Anomalies')
plt.title('Time Series Anomaly Detection using Isolation Forest (PyOD)')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
print(classification_report(df['true_labels'], y_pred_if))


In [ ]:
accuracy = accuracy_score(df['true_labels'], y_pred_if)
accuracy*100

## HBOS

In [ ]:
clf_hbos = HBOS()
clf_hbos.fit(X)

In [ ]:
y_predict_hbos = clf_hbos.predict(X)

In [ ]:
y_scores_hbos=clf_hbos.decision_function(X)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['timestamp'], df['value'], label='Time Series Data')
plt.scatter(df['timestamp'][y_predict_hbos == 1], df['value'][y_predict_hbos == 1], 
            color='red', marker='x', label='Detected Anomalies (HBOS)')
plt.scatter(df['timestamp'][df['true_labels'] == 1], df['value'][df['true_labels'] == 1], 
            color='blue', marker='o', label='Ground Truth Anomalies')
plt.title('Time Series Anomaly Detection using HBOS (PyOD)')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
print(classification_report(df['true_labels'], y_predict_hbos ))


In [ ]:
accuracy = accuracy_score(df['true_labels'], y_predict_hbos)
accuracy*100